In [176]:
 import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas

In [177]:
csvpath = Path('../Project-03/teamGameStats.csv')
teamstats_df = pd.read_csv(csvpath)

In [178]:
teamstats_df.info('include=all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13658 entries, 0 to 13657
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TEAM      13658 non-null  object 
 1   MATCHUP   13658 non-null  object 
 2   GAMEDATE  13658 non-null  object 
 3   W/L       13658 non-null  object 
 4   MIN       13658 non-null  int64  
 5   PTS       13658 non-null  int64  
 6   FGM       13658 non-null  int64  
 7   FGA       13658 non-null  int64  
 8   FG%       13658 non-null  float64
 9   3PM       13658 non-null  int64  
 10  3PA       13658 non-null  int64  
 11  3P%       13658 non-null  float64
 12  FTM       13658 non-null  int64  
 13  FTA       13658 non-null  int64  
 14  FT%       13658 non-null  float64
 15  OREB      13658 non-null  int64  
 16  DREB      13658 non-null  int64  
 17  REB       13658 non-null  int64  
 18  AST       13658 non-null  int64  
 19  TOV       13658 non-null  int64  
 20  STL       13658 non-null  in

In [179]:
teamstats_df = teamstats_df.drop(columns=['TEAM','MATCHUP','GAMEDATE', 'W/L','+/-','PTS'])

In [180]:
#teamstats_scaled = StandardScaler().fit_transform(teamstats_df)
#print(teamstats_scaled[0:5])

In [181]:
# defining 'W/L' column as feature set
#X = teamstats_df.copy()
#X.drop('W/L', axis=1, inplace=True)
#X.head(20)

In [182]:
# converting 'W/L' to 0/1's
#y=teamstats_df['W/L'].apply(lambda x: 1 if x=="W" else 0)


In [183]:
pca = PCA(n_components=10)

#teamstats_pca = pca.fit_transform(teamstats_scaled)
teamstats_pca = pca.fit_transform(teamstats_df)

In [184]:
df_teamstats_pca = pd.DataFrame(data=teamstats_pca, columns=['pc01', 'pc02', 'pc03', 'pc04', 'pc05', 'pc06', 'pc07', 'pc08', 'pc09', 'pc10'])#, 'pc11', 'pc12', 'pc13', 'pc14', 'pc15', 'pc16', 'pc17', 'pc18', 'pc19', 'pc20'])
df_teamstats_pca.head()

,pc01,pc02,pc03,pc04,pc05,pc06,pc07,pc08,pc09,pc10
0,-1.099512,-3.006163,5.632135,16.110408,-1.191430,3.834534,3.862737,-4.882819,0.775267,5.984189
1,-12.602726,1.067494,3.406087,0.247030,7.882332,-10.621052,-4.151002,3.088798,4.776194,-0.818932
2,13.215689,4.741486,16.265586,4.715740,-5.181478,11.446973,4.059616,-3.642305,-7.042901,-0.409916
3,-7.656956,-0.842904,9.717533,3.059729,2.036471,-9.280019,1.755001,-3.728750,-9.683466,-0.670320
4,9.234957,11.658581,4.708058,4.683908,7.057334,-0.046740,3.474690,-6.055132,1.040679,2.314814


In [185]:
pca.explained_variance_ratio_

array([0.20249926, 0.1864045 , 0.16012383, 0.13058555, 0.09311634,
       0.07006045, 0.04952517, 0.03054593, 0.02247834, 0.01768099])

In [188]:
inertia = []
k = list(range(1,11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_teamstats_pca)
    inertia.append(km.inertia_)
    

# Creating the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [191]:
 # Predicting clusters with k2

# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_teamstats_pca)

# Predict clusters
predictions = model.predict(df_teamstats_pca)

# Add the predicted class columns
df_teamstats_pca["class"] = model.labels_
df_teamstats_pca.head()

,pc01,pc02,pc03,pc04,pc05,pc06,pc07,pc08,pc09,pc10,class
0,-1.099512,-3.006163,5.632135,16.110408,-1.191430,3.834534,3.862737,-4.882819,0.775267,5.984189,3
1,-12.602726,1.067494,3.406087,0.247030,7.882332,-10.621052,-4.151002,3.088798,4.776194,-0.818932,3
2,13.215689,4.741486,16.265586,4.715740,-5.181478,11.446973,4.059616,-3.642305,-7.042901,-0.409916,0
3,-7.656956,-0.842904,9.717533,3.059729,2.036471,-9.280019,1.755001,-3.728750,-9.683466,-0.670320,3
4,9.234957,11.658581,4.708058,4.683908,7.057334,-0.046740,3.474690,-6.055132,1.040679,2.314814,1


In [190]:
# Plotting the clusters
df_teamstats_pca.hvplot.scatter(
    x="pc01",
    y="pc02",
    hover_cols=["class"],
    by="class",
)

:NdOverlay   [class]
   :Scatter   [pc01]   (pc02,class)